# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_original = pd.read_csv("../output_data/cities1.csv")
city_data_original.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Bluff,-46.6000,168.3333,46.99,84,96,21.00,NZ,1619402292
1,Sayyan,15.1718,44.3244,59.41,46,94,0.83,YE,1619402292
2,Yellowknife,62.4560,-114.3525,10.40,72,90,12.66,CA,1619402293
3,Rey Bouba,8.6724,14.1786,83.53,35,30,2.91,CM,1619402293
4,Cravo Norte,6.3017,-70.2041,87.96,46,100,2.10,CO,1619402294


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [34]:
# Store latitude and longitude in locations
humidity_map = city_data_original[["Lat","Lng"]]

humidity_values = city_data_original["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(humidity_map, weights=humidity_values, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
'''
* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

'''

In [28]:
narrowed_df = city_data_original[((city_data_original['Max_Temp'] < 80) & (city_data_original['Max_Temp'] > 70)) &
                                (city_data_original['Wind_Speed'] < 10) &
                                 (city_data_original['Cloudiness'] == 0)]
narrowed_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
75,Kiama,-34.6833,150.8667,73.00,54,0,1.99,AU,1619402337
128,Chaman,30.9236,66.4512,71.02,11,0,5.44,PK,1619402367
162,Vila Velha,-20.3297,-40.2925,77.00,83,0,8.05,BR,1619402387
176,Koungou,-12.7336,45.2042,77.00,88,0,5.75,YT,1619402394
232,Carnarvon,-24.8667,113.6333,78.80,31,0,6.91,AU,1619402289
261,Bouillante,16.1304,-61.7692,75.20,78,0,2.30,GP,1619402450
268,Maceió,-9.6658,-35.7353,73.40,94,0,2.30,BR,1619402454
439,Flinders,-34.5833,150.8552,73.00,53,0,1.01,AU,1619402559
459,Nālāgarh,31.0500,76.7167,76.60,25,0,4.94,IN,1619402570
481,Boueni,-12.9025,45.0761,77.00,88,0,5.75,YT,1619402582


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels within 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
hotel_df = narrowed_df

params = {
    "radius": 5000,
    "key": g_key,
    "types": "lodging",
}


hotel_df['Lat']
hotel_df['Lat']

for index, row in hotel_df.iterrows():

    lat=row['Lat']
    lng=row['Lng']

    params['location'] = f"{lat}, {lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address['results'][0]['name']
    except:
        print('Missing field/result... skipping')

    coordinates = hotel_df[["Lat","Lng"]]

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,map_type='SATELLITE')

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)

# Add the layer to the map
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

C:\Users\17147\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [38]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
75,Kiama,-34.6833,150.8667,73.00,54,0,1.99,AU,1619402337,The Sebel Kiama Harbourside
128,Chaman,30.9236,66.4512,71.02,11,0,5.44,PK,1619402367,Niamat parda Center chaman
162,Vila Velha,-20.3297,-40.2925,77.00,83,0,8.05,BR,1619402387,Quality Suites Vila Velha
176,Koungou,-12.7336,45.2042,77.00,88,0,5.75,YT,1619402394,Villa Netibor
232,Carnarvon,-24.8667,113.6333,78.80,31,0,6.91,AU,1619402289,Hospitality Carnarvon
261,Bouillante,16.1304,-61.7692,75.20,78,0,2.30,GP,1619402450,Gîtes Dardanelle
268,Maceió,-9.6658,-35.7353,73.40,94,0,2.30,BR,1619402454,Poutur Pousada
439,Flinders,-34.5833,150.8552,73.00,53,0,1.01,AU,1619402559,Shellharbour Resort & Conference Centre
459,Nālāgarh,31.0500,76.7167,76.60,25,0,4.94,IN,1619402570,Fort Nalagarh
481,Boueni,-12.9025,45.0761,77.00,88,0,5.75,YT,1619402582,The feet in water


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index1, row1 in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# Add marker layer ontop of heat map
fig1 = gmaps.figure(layout=figure_layout)

markers1 = gmaps.marker_layer(locations)

fig1.add_layer(heat_layer)
fig1.add_layer(markers1)
# Display figure
fig1

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…